In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
# from linearmodels.panel.data import PanelData
# from linearmodels.panel import PanelOLS, PooledOLS, RandomEffects, compare
import matplotlib.pyplot as plt
import pyblp

In [3]:
product_data=pd.read_csv("../data/merged/len3_ndb_agg_blp_imputed_lags.csv",encoding="shift-jis",index_col=0)
product_data["market_ids"]=product_data["薬効分類"].astype(int).astype(str)+"-"+product_data["year"].astype(str)
product_data["generic_share_q"].fillna(0,inplace=True)
product_data["generic_share_r"].fillna(0,inplace=True)
product_data.rename(columns={"薬価":"prices","r_share":"shares",
                    # "elasped_0":"demand_instruments0",
                    # "elasped_1":"demand_instruments1",
                    # 'elasped_2': 'demand_instruments2',
                    # 'elasped_3': 'demand_instruments3',
                    # 'elasped_4': 'demand_instruments4',
                    # 'elasped_5': 'demand_instruments5',
                    # 'elasped_6': 'demand_instruments6',
                    # 'elasped_7': 'demand_instruments7',
                    # 'elasped_8': 'demand_instruments8',
                    # 'elasped_9': 'demand_instruments9',
                    # "year":"demand_instruments2",
                    # "generic_per":"demand_instruments4",
                    "mean_price":"demand_instruments2",
                    # "otc_dom":"demand_instruments2",
                    "lag_mean_price":"demand_instruments0",
                    # "generic_share_q":"demand_instruments7",
                    "generic_share_r":"demand_instruments1"
                    },inplace=True)
product_data=product_data.loc[product_data["shares"]>0]
# data["shares"]=data["r_share"]
product_data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,prices,後発品区分,総計,year,in_hospital,...,id_441.0,id_449.0,id_625.0,id_629.0,id_799.0,q_share,shares,generic_share_q,demand_instruments1,market_ids
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014.0,1,...,0.0,0.0,0.0,0.0,0.0,0.000013,0.000007,0.242981,0.053692,114-2014.0
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014.0,1,...,0.0,0.0,0.0,0.0,0.0,0.000006,0.000003,0.242981,0.053692,114-2014.0
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014.0,1,...,0.0,0.0,0.0,0.0,0.0,0.000005,0.000003,0.242981,0.053692,114-2014.0
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014.0,0,...,0.0,0.0,0.0,0.0,0.0,0.000029,0.000016,0.242981,0.053692,114-2014.0
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014.0,0,...,0.0,0.0,0.0,0.0,0.0,0.000018,0.000010,0.242981,0.053692,114-2014.0


In [4]:
logit_formulation = pyblp.Formulation('prices+oral+in_hospital+後発品区分+year', absorb='C(薬効分類)')
logit_formulation

prices + oral + in_hospital + 後発品区分 + year + Absorb[C(薬効分類)]

In [5]:
# product_data["shares"]-=1e-10
problem = pyblp.Problem(logit_formulation, product_data)
problem

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T     N      K1    MD    ED 
---  ------  ----  ----  ----
264  104614   5     7     1  

Formulations:
     Column Indices:          0      1         2         3     4  
--------------------------  ------  ----  -----------  -----  ----
X1: Linear Characteristics  prices  oral  in_hospital  後発品区分  year


Dimensions:
 T     N      K1    MD    ED 
---  ------  ----  ----  ----
264  104614   5     7     1  

Formulations:
     Column Indices:          0      1         2         3     4  
--------------------------  ------  ----  -----------  -----  ----
X1: Linear Characteristics  prices  oral  in_hospital  後発品区分  year

In [6]:
logit_results = problem.solve()
logit_results

Solving the problem ...
Updating the weighting matrix ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix
Step      Value      Shares   Condition Number
----  -------------  -------  ----------------
 1    +5.265806E+02     0      +4.723767E+06  

Estimating standard errors ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 2    +7.336639E+01     0      +6.754170E+07      +1.415781E+07  

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
    prices            oral          in_hospital         後発品区分            year      
---------------  ---------------  ---------------  ---------------  ---------------
 -2

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 2    +7.336639E+01     0      +6.754170E+07      +1.415781E+07  

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
    prices            oral          in_hospital         後発品区分            year      
---------------  ---------------  ---------------  ---------------  ---------------
 -2.506787E-03    +2.293815E+00    +2.753195E-01    -2.318181E+00    -2.183268E-02 
(+1.189059E-03)  (+1.069333E-01)  (+3.019719E-02)  (+3.515455E-01)  (+1.001821E-02)